In [1]:
import torch 
import torch.nn as nn  
import numpy as np
from tqdm.auto import tqdm

import cv2
import os,sys
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from torch.cuda.amp import autocast
from torch.cuda import amp
from torch.optim import lr_scheduler
import time
import gc
import copy
from collections import defaultdict
import torch.nn.functional as F
import matplotlib.pyplot as plt
import albumentations as A
from pathlib import Path
from PIL import Image
import segmentation_models_pytorch as smp
import time

from segmentation.scr.utils.metrics import dice_coef

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel



#from segmentation.config import Configs as CFG
from segmentation.models.unet import unet
from segmentation.scr.utils import losses, metrics, transforms, rle_coding
from segmentation.scr.utils.utils import set_seed
from segmentation.scr.tilling_dataset import Tilling_Dataset
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

#from segmentation.scr
pd.options.mode.chained_assignment = None

c:\Users\123\Desktop\human vasculature\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\123\AppData\Local\Temp\ipykernel_24608\2019418347.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [14]:
class CFG:

    # configs for tilling dataset kidney 1
    path_img_kidney1 = "data\\train\\kidney_1_dense\\images"
    path_lb_kidney1 = "data\\train\\kidney_1_dense\\labels"
    path_df_kidney_1_till = "data\\kidney_1_tilling.csv"
    tile_size = (256, 256)
    overlap_pct = 0
    cache_dir = "data"
    mean_till1 = (0.2425, 0.2425, 0.2425)
    std_till1 = (0.1766, 0.1766, 0.1766)

    # configs for tilling dataset kidney 3
    path_img_kidney3 = "data\\train\\kidney_3_sparse\\images"
    path_lb_kidney3 = "data\\train\\kidney_3_dense\\labels"
    path_df_kidney_3_till = "data\\kidney_3_tilling.csv"
    tile_size = (256, 256)
    overlap_pct = 0
    cache_dir = "data"

    # configs for transforms
    p_rot = 0.3
    p_aug = 0.3
    # cofigs for train / eval model
    random_seed = 0
    device = "cuda" if torch.cuda.is_available() else "cpu"
    train_batch_size = 2
    n_accumulate = max(1, 8 // train_batch_size)
    valid_batch_size = train_batch_size * 2
    clip_norm=5
    epochs = 28
    lr = 3e-4
    dice_th = 0.5
    
    
    

In [15]:
train_transform  = transforms.get_transform(transform_type='strong')
val_transform = transforms.get_transform(transform_type='val')

In [16]:
train_dataset = Tilling_Dataset(
    name_data='kidney_1_tilling',
    path_to_df=CFG.path_df_kidney_1_till,
    use_random_sub=True,
    empty_tile_pct=0,
    sample_limit=160,
    random_seed=42
)


160
Dataset contains 20744 empty and 33952 non-empty tiles.
Sample 0 empty and 160 non-empty tiles.


In [17]:
val_dataset = Tilling_Dataset(
    name_data='kidney_3_tilling',
    path_to_df=CFG.path_df_kidney_3_till,
    use_random_sub=True,
    empty_tile_pct=10,
    sample_limit=50,
    random_seed=42)

50
Dataset contains 14595 empty and 6447 non-empty tiles.
Sample 5 empty and 45 non-empty tiles.


In [18]:
set_seed.set_seed(CFG.random_seed)

train_loader = DataLoader(train_dataset, batch_size=CFG.train_batch_size, num_workers=2, shuffle=True, pin_memory=True)
val_loader = DataLoader(train_dataset, batch_size=CFG.valid_batch_size, num_workers=2, shuffle=False, pin_memory=True)

In [30]:
model = unet.UNet(n_channels=3, n_classes=1, bilinear=False).to(CFG.device)
num_epoch = 4
loss_fn = losses.BCE_DICE(mode="BCE")
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
device = CFG.device

In [31]:
def train_one_loop(model, optimizer, loss_func, train_loader, device=CFG.device, grad_clip = False):
    model.train()
    running_loss = 0.0
    epoch_loss = 0.0

    dataset_size = 0
    optimizer.zero_grad()
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc='Train ')
    for step, batch in pbar:
        images, masks,_, _ = batch
        images = images.to(device, dtype=torch.float)
        masks  = masks.to(device, dtype=torch.float)
        
        batch_size = images.shape[0]
        dataset_size += batch_size
    
        y_pred = model(images)
            
        loss   = loss_func(y_pred, masks)
        loss   = loss / CFG.n_accumulate
        loss.backward() #loss.backward()  # backward-pass

        running_loss += loss.item() * batch_size
        dataset_size += batch_size
        del images
        del masks
        del y_pred
        
        if (step + 1) % CFG.n_accumulate == 0 or (step + 1 == len(train_loader)):
            if grad_clip:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=CFG.clip_norm)
            optimizer.step()  # update weights
            optimizer.zero_grad()



        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        current_lr = optimizer.param_groups[0]['lr']
        pbar.set_postfix( epoch=f'{step + 1}',
                          train_loss=f'{running_loss / dataset_size:0.4f}',
                          #train_dice = f'{train_dice:0.4f}',
                          #train_jaccard = f'{train_jaccard:0.4f}',
                          lr=f'{current_lr:0.5f}',
                          gpu_mem=f'{mem:0.2f} GB')
    epoch_loss = running_loss / dataset_size

    gc.collect()
    torch.cuda.empty_cache()
    
    return epoch_loss
        

In [32]:
train_one_loop(model=model,optimizer=optimizer, train_loader=train_loader,
               loss_func=loss_fn, device=CFG.device)

Train : 100%|██████████| 80/80 [00:16<00:00,  4.79it/s, epoch=80, gpu_mem=1.69 GB, lr=0.00010, train_loss=0.0716]


0.07155871191062033

In [33]:
def valid_one_epoch(model, dataloader, loss_fn,  epoch, device=CFG.device):
 #(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0.0
    val_scores = []
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for _, batch in pbar:
        images, masks,_, _ = batch 
        images  = images.to(device, dtype=torch.float)
        masks   = masks.to(device, dtype=torch.float)

        batch_size = images.size(0)
        with torch.no_grad():
            
            y_pred  = model(images)
            loss    = loss_fn(y_pred, masks)

        running_loss += loss.item() * batch_size
        dataset_size += batch_size

        

        
        val_dice = dice_coef(y_pred=y_pred, y_true=masks).cpu().detach().numpy()
        val_scores.append([val_dice])
        del(images)
        del(masks)
        del(y_pred)
        mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
        pbar.set_postfix(valid_loss=f'{running_loss / dataset_size:0.4f}',
                        gpu_memory=f'{mem:0.2f} GB')
    val_score  = np.mean(val_scores, axis=0)
    epoch_loss = running_loss / dataset_size
    
    torch.cuda.empty_cache()
    gc.collect()
    return epoch_loss, val_score

In [34]:
valid_one_epoch(model=model, loss_fn=loss_fn, dataloader=val_loader,epoch=0)

Valid : 100%|██████████| 40/40 [00:08<00:00,  4.55it/s, gpu_memory=1.09 GB, valid_loss=0.4406]


(0.4405678234994411, array([0.02517452], dtype=float32))

In [ ]:
def train_model(model, 
                optimizer, 
                loss_func, 
                train_loader, 
                val_loader, 
                num_epochs, 
                scheduler=None, 
                device =CFG.device,
                path_to_save=CFG.path_to_save_state_model,
):
    for epoch in range(1, num_epochs + 1):

        print(f'Epoch {epoch}/{num_epochs}')
        print('-' * 10)